# Erstellen einer FHIR Patient Ressource mit dem Python fhirclient
Um den Python fhirclient zu installieren kann man entweder die aktuellste Version aus dem [GitHub Repository](https://github.com/smart-on-fhir/client-py) nehmen oder die zuletzt auf [PyPi](https://pypi.org/project/fhirclient/) veröffentlichte Version. Da diese jedoch noch STU-3 ist, nutzen wir für die folgenden Beispiele die GitHub Version (R4).

In [ ]:
!pip install git+git://github.com/smart-on-fhir/client-py.git

## IFrame wird hier nur zu Darstellungszwecken importiert und wird nicht für die späteren Aufgaben benötigt.

In [ ]:
from IPython.display import IFrame

In [ ]:
# Patient Ressource in der FHIR Spezifikation.
IFrame('https://hl7.org/fhir/patient.html#resource', width=1200, height=330)

In [ ]:
from fhirclient.models import patient

In [ ]:
# Instanziieren eines Patienten
my_patient = patient.Patient()

In [ ]:
# help() Funktion, um mehr Informationen zu erhalten
help(my_patient)

## Mit der Patient Ressource vertraut machen
Um erstmal einen Überblick davon zu erhalten wie die Patienten Ressource aufgebaut ist, bzw. welche Attribute diese enthält kann die Funktion **elementProperties()** aufgerufen werden, diese gibt dann alle Attribute der fhirclient Klasse **Patient()** mit ihren jeweiligen Eigenschaften zurück.

In [ ]:
for property in my_patient.elementProperties():
    print(property)

## Name des Patienten / der Patientin

In [ ]:
IFrame('https://hl7.org/fhir/datatypes.html#HumanName', width=1200, height=330)

In [ ]:
from fhirclient.models import humanname

In [ ]:
name = humanname.HumanName()
name.family = 'Bach'
name.given = ['Johann', 'Sebastian']
name.text = 'Johann Sebastian Bach'
name.use = 'official'
my_patient.name = [name]

## Geburtsdatum

In [ ]:
IFrame('https://hl7.org/fhir/datatypes.html#date', width=1200, height=330)

In [ ]:
from fhirclient.models import fhirdate

In [ ]:
date_of_birth = fhirdate.FHIRDate('1967-05-25')
my_patient.birthDate = dateOfBirth

In [ ]:
import json

In [ ]:
print(json.dumps(my_patient.as_json(), indent=4))

## Pythons Requests Library
Die [Requests Library](https://requests.readthedocs.io/en/master/) enthält Funktionen um HTTP Anfragen schnell um simpel an einen Server zu schicken. 

In [ ]:
import requests

In [ ]:
fhir_test_server = 'https://server.fire.ly'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

In [ ]:
response = requests.post(f'{fhir_test_server}/Patient/$validate', headers = headers, data = json.dumps(my_patient.as_json()))
operation_outcome = response.json()

In [ ]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [ ]:
print(json.dumps(operation_outcome, indent=4))

In [ ]:
response = requests.post(f'{fhir_test_server}/Patient', headers = headers, data = json.dumps(my_patient.as_json()))
operation_outcome = response.json()

In [ ]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [ ]:
print(json.dumps(operation_outcome, indent=4))